In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
from scipy import interpolate
import os
from astropy.time import Time
import pandas as pd


#Function to convert the data into a usable format
def ImportData( Bands , MAGArray , TypeArray , MaskFilter) :

    count0 = 0
    while count0 < len(Bands) :
        count1 = 0
        while count1 < Bands[count0].size :
            if TypeArray[count1] == MaskFilter[count0] :
                Bands[count0][count1] = MAGArray[count1]
            else:
                Bands[count0][count1] = None
            count1 = count1 + 1
        count0 = count0 + 1

    return Bands

#finds Opposition Data using Equation
def OppDateFind( H , M , S ) :

    return datetime.timedelta( days = 365.25/24 * ( H + M/60 + S/3600 ) ).total_seconds() / 3600 / 24\

#Function to process the Quasar Data
def interpQuasar( DATA  , OppDate):

    #Load Data into an Array
    output = np.loadtxt( DATA , skiprows = 2 ,  delimiter = ',' , unpack = 1 , dtype = str )

    #Separate the Data into its Components
    Type = output[4]
    MAG = output[2].astype('float')
    ERR = output[3].astype('float')
    MJD = output[1].astype('float')

    #Variable stores number of data sets
    n = MJD.size

    #Create empty arrays with a size equal to the number of data sets for each band
    Bands = [ u , g , r , i , i2 , z ] = [ np.empty(n) , np.empty(n) , np.empty(n) , np.empty(n) , np.empty(n) , np.empty(n) ]
    Bands_ERR = [ u_ERR , g_ERR , r_ERR , i_ERR , i2_ERR ,  z_ERR ] = [ np.empty(n) , np.empty(n) , np.empty(n) , np.empty(n) , np.empty(n) , np.empty(n) ]
    Filter = [ 'U' , 'G' , 'R' , 'I' , 'I2' , 'Z' ]

    #Import the data for the bands
    Bands = ImportData( Bands , MAG , Type , Filter )

    #Shifts the i2 band
    i2 = i2 + 0.04*( np.median(i[~np.isnan(i)]) - np.median(g[~np.isnan(g)]) ) + 0.076

    #Loop through the data and set the "i" and "i2" values to 0 if they are null. I think this is to avoid an error when adding the values of i2 to the values of i
    count0 = 0
    while count0 < n :
        if np.isnan(i)[count0] == True :
            i[count0] = 0
        if np.isnan(i2)[count0] == True :
            i2[count0] = 0
        count0 = count0 + 1

    #Adding i2 to i to get a single i band
    i = i2 + i


    #Set the values in the i array that were previously null back to null
    count0 = 0
    while count0 < n :
        if i[count0] == 0 :
            i[count0] = None
        count0 = count0 + 1

    #Shifting MJD
    mjd = MJD + 61 - 365.25*int(np.min( MJD + 61 )/365.25) - OppDate - 7

    #Arrays to store the data for the bands
    Bands = [ u , g , r , i , z ]
    #Array to store the monthly median brightnesses for each band
    MonthMedian = [ np.empty(0) , np.empty(0) , np.empty(0) , np.empty(0) , np.empty(0) ]

    #Days in a Typical Lunar Month
    time = 29.53

    #Storing the median brightness for each band for each month in the MonthMedian Array
    count0 = 0
    while count0 < len(MonthMedian) :
        count1 = 0
        count2 = 0
        #3650/time refers to the number of months in 10 years
        #This is thus looping through the data month-by-month to get monthly medians
        while count1 < 3650/time :
            temp = []
            #Gets the data for the month for the median calculation
            #Gets data from the middle of a month to the middle of the next month
            while count2 < mjd.size and mjd[count2] < time * ( count1 - 1/2 ) :
                #Stores the data in a temoorary array
                if np.isnan(Bands[count0])[count2] == 0 :
                    temp = np.append( temp , Bands[count0][count2] )
                count2 = count2 + 1
            #Appends the Month Median for the band at the end of the respective array
            MonthMedian[count0] = np.append( MonthMedian[count0] , np.median(temp) )
            count1 = count1 + 1
        count0 = count0 + 1

    [ U , G , R , I , Z ] = MonthMedian

    YearMedian = [ [ [] , [] , [] , [] , [] , [] , [] , [] , [] , [] ] ,
                  [ [] , [] , [] , [] , [] , [] , [] , [] , [] , [] ] ,
                  [ [] , [] , [] , [] , [] , [] , [] , [] , [] , [] ] ,
                  [ [] , [] , [] , [] , [] , [] , [] , [] , [] , [] ] ,
                  [ [] , [] , [] , [] , [] , [] , [] , [] , [] , [] ]
                 ]

    #Initialises the monthly median values in YearMedian in the format: YearMedian[Band][Year Number][Month Number]
    count0 = 0
    while count0 < len(MonthMedian) :
        count1 = 0
        while count1 < MonthMedian[count0].size :
            YearMedian[count0][int(count1/(365.25/29.53))] = np.append( YearMedian[count0][int(count1/(365.25/29.53))] , MonthMedian[count0][count1] )
            count1 = count1 + 1
        count0 = count0 + 1


    #Changes YearMedian so that it stores the annual medians by calculating the median for each year, ignoring the null values
    count0 = 0
    while count0 < len(YearMedian) :
        count1 = 0
        while count1 < len(YearMedian[count0]) :
            YearMedian[count0][count1] = np.median( YearMedian[count0][count1][~np.isnan(YearMedian[count0][count1])] )
            count1 = count1 + 1
        count0 = count0 + 1


    #Shifting Function that removes the effect of the band
    TEMP = [ np.empty(mjd.size) , np.empty(mjd.size) , np.empty(mjd.size) , np.empty(mjd.size) , np.empty(mjd.size) ]
    count0 = 0
    while count0 < len(TEMP) :
        count1 = 0
        count2 = 0
        while count1 < 3650/time :
            #Gets data from the middle of a month to the middle of the next month
            while count2 < mjd.size and mjd[count2] < time * ( count1 - 1/2 ) :
                if np.isnan( MonthMedian[count0][count1] - MonthMedian[1][count1] ) == 0 :
                    TEMP[count0][count2] = Bands[count0][count2] - ( MonthMedian[count0][count1] - MonthMedian[1][count1] )
                else :
                    TEMP[count0][count2] = Bands[count0][count2] - ( MonthMedian[count0][count1] - MonthMedian[1][count1] )
                count2 = count2 + 1
            count1 = count1 + 1
        count0 = count0 + 1

    [ you , gee , are , eye , zee ] = TEMP

    Total = np.empty(mjd.size)

    Total[:] = np.nan

    #Reduces the size of TEMP
    count0 = 0
    while count0 < len(TEMP) :
        count1 = 0
        while count1 < mjd.size :
            if np.isnan(TEMP[count0])[count1] == 0 :
                Total[count1] = TEMP[count0][count1]
            count1 = count1 + 1
        count0 = count0 + 1

    #Calculates the monthly medians for the Total Brightness
    count0 = 0
    mjd = MJD + 61 - 365.25*int(np.min( MJD + 61 )/365.25) - OppDate - 7
    total = []
    count1 = 0
    count2 = 0
    while count1 < 3650/time :
        temp = []
        while count2 < Total.size and mjd[count2] < time * ( count1 - 1/2 ) :
            if np.isnan(Total)[count2] == 0 :
                temp = np.append( temp , Total[count2] )
            count2 = count2 + 1
        total = np.append( total , np.median(temp) )
        count1 = count1 + 1

    #Array that stores the mjd of 124 months
    mjd2 = np.linspace( -1*29.53 , 122*29.53 , num = 124 )

    #Array to store an mjd value for every value in Total
    #These are equally spaced in 124 months
    mjd3 = np.linspace( np.min(mjd2[~np.isnan(total)]) , np.max(mjd2[~np.isnan(total)]) , Total.size )

    #Adds a magnitude value for every mjd value for mjd3 and stores it in an Array
    INTERPOL = interpolate.interp1d( mjd2[~np.isnan(total)] , total[~np.isnan(total)] )( mjd3 )

    #Arrays to divide the values into the 6 epochs where data is gathered
    #This is to remove the data from the parts where there is no reliable data available
    INTERPOLDivided = [i1,i2,i3,i4,i5,i6] = [[],[],[],[],[],[]]

    mjdDivided = [m1,m2,m3,m4,m5,m6] = [[],[],[],[],[],[]]
    mjd2Divided = [m21,m22,m23,m24,m25,m26] = [[],[],[],[],[],[]]
    mjd3Divided = [m31,m32,m33,m34,m35,m36] = [[],[],[],[],[],[]]

    TotalDivided = [T1,T2,T3,T4,T5,T6] = [[],[],[],[],[],[]]
    totalDivided = [t1,t2,t3,t4,t5,t6] = [[],[],[],[],[],[]]

    #An array to store the start and end mjd values for the 6 epochs where data is gathered
    ranges = [(-40, 65), (320, 445), (625, 830), (975, 1185), (1350, 1510), (1745, 1870)]


    #Initilisies mjdDivided and TotalDivided
    for date, data in zip(mjd, Total):
        for count, (start, end) in enumerate(ranges):
            if start <= date <= end:
                mjdDivided[count].append(date)
                TotalDivided[count].append(data)
                break

    #Initilisies mjd2Divided and totalDivided
    for date, data in zip(mjd2, total):
        for count, (start, end) in enumerate(ranges):
            if start <= date <= end:
                mjd2Divided[count].append(date)
                totalDivided[count].append(data)
                break

    #Initialises mjd3Divided
    for count in range(6):
        mjd2Divided_array = np.array(mjd2Divided[count])
        if mjd2Divided_array[~np.isnan(totalDivided[count])].size > 0:
            mjd3Divided[count] = np.linspace( np.min(mjd2Divided_array[~np.isnan(totalDivided[count])]) , np.max(mjd2Divided_array[~np.isnan(totalDivided[count])]) , len(TotalDivided[count]))

    #Initialises INTERPOLDivided
    for count in range(len(INTERPOLDivided)):
        mjd2Divided_array = np.array(mjd2Divided[count])
        totalDivided_array = np.array(totalDivided[count])
        if mjd2Divided_array[~np.isnan(totalDivided_array)].size > 1 and totalDivided_array[~np.isnan(totalDivided_array)].size > 1:
            INTERPOLDivided[count] = interpolate.interp1d( mjd2Divided_array[~np.isnan(totalDivided_array)] , totalDivided_array[~np.isnan(totalDivided_array)] )( mjd3Divided[count] )

    #Concatenates the divided arrays back into single arrays
    mjd3 = np.array(np.concatenate(mjd3Divided))
    INTERPOL = np.array(np.concatenate(INTERPOLDivided))

    return INTERPOL, mjd3, YearMedian, MonthMedian, mjd, Bands, MAG

#Function that creates the data files
def FUNC( DATA , select , YearShift , OppDate, D_Num, i) :

    #Number of Days in a lunar month
    time = 29.53

    #Does everything done to the Quasar to the RR Lyrae
    output = np.loadtxt( 'TargetData.csv' , skiprows = 1 , delimiter = ',' , unpack = 1 , dtype = 'str' )

    Target_ID = output[0].astype('float')
    Target_Type = output[1]
    Target_MAG = output[7].astype('float')
    Target_MJD = output[9].astype('float')

    n = Target_MJD.size

    Target_mjd = Target_MJD + 61 - 365.25*int(np.min( Target_MJD + 61 )/365.25) - OppDateFind(12,30,49) + YearShift*365 + 7

    Target_Bands = [ Target_u , Target_g , Target_r , Target_i , Target_z ] = [ np.empty(n) , np.empty(n) , np.empty(n) , np.empty(n) , np.empty(n) ]

    Target_Filter = [ 'u' , 'g' , 'r' , 'i' , 'z' ]

    Target_Bands = ImportData( Target_Bands , Target_MAG , Target_Type , Target_Filter )

    Target_Bands2 = []

    #If a particular Data Set is Selected, make all other Target Data null
    if select != 0 :
        count0 = 0
        while count0 < len(Target_Bands) :
            count1 = 0
            while count1 < n :
                if Target_ID[count1] != select :
                    Target_Bands[count0][count1] = None
                count1 = count1 + 1
            count0 = count0 + 1
        count0 = 0

        while count0 < n :
            if Target_ID[count0] != select :
                Target_MAG[count0] = None
                Target_Bands2 = np.append(Target_Bands2, None)
                count0 = count0+1
                continue
            Target_Bands2 = np.append(Target_Bands2, Target_Type[count0])
            count0 = count0 + 1


    #Get all indices that are not null
    index = []
    count0 = 0
    while count0 < INTERPOL.size :
        if np.isnan(INTERPOL)[count0] == 0 :
            index.append(count0)
        count0 = count0 + 1

    Target_index = []
    count0 = 0
    while count0 < Target_mjd.size :
        if np.isnan(Target_MAG)[count0] == 0:
            Target_index.append(count0)
        count0 = count0 + 1
    Target_dates = Target_mjd[Target_index]

    #Raise ValueError if a data point is outside the 6 epochs
    if(max(Target_dates) > 1510 or min(Target_dates) < min(mjd3)):
        raise ValueError

    #Get Quasar Data Points that are closest to each RR Lyrae Data Point
    count0 = 0
    selector = []
    while count0 < Target_dates.size :
        DiffArr = np.abs( mjd3[index] - Target_dates[count0] )
        count1 = 0
        while count1 < mjd3[index].size :
            if DiffArr[count1] == np.min(DiffArr) :
                selector.append(index[count1])
                index = np.delete( index , count1 )
            count1 = count1 + 1
        count0 = count0 + 1

    #Separates the data based on the Bands
    #The Band data is taken from the RR Lyrae Data
    Final = []
    Final_Bands = []
    Final_Dates = []
    count2 = 0
    while count2 < len(Target_Filter) :
        count0 = 0
        count1 = 0
        while count1 < 3650/time :
            while count0 < INTERPOL[selector].size and mjd3[selector][count0] < time * ( count1 - 1/2 ) :
                if Target_Bands2[Target_index][count0] == Target_Filter[count2]:
                    if np.isnan( g_med[count1] - MonthMedian[count2][count1] ) == True :
                        Final = np.append( Final , INTERPOL[selector][count0] - ( YearMedian[1][int(count1/(365.25/29.53))] - YearMedian[count2][int(count1/(365.25/29.53))] ) )
                    else :
                        Final = np.append( Final , INTERPOL[selector][count0] - ( g_med[count1] - MonthMedian[count2][count1] ) )
                    Final_Bands = np.append(Final_Bands, count2)
                    Final_Dates = np.append(Final_Dates, mjd3[selector][count0])
                count0 = count0 + 1
            count1 = count1 + 1
        count2 = count2 + 1
    Final = np.array( Final )

    [ a , b , c , d , e ] = [ np.empty( len(Final) ) , np.empty( len(Final) ) , np.empty( len(Final) ) , np.empty( len(Final) ) , np.empty( len(Final) ) ]

    BAMDS = [ a , b , c , d , e ]
    count0 = 0
    while count0 < len( [ a , b , c , d , e ] ) :
        count1 = 0
        while count1 < len(Final) :
            if Final_Bands[count1] == count0 :
                BAMDS[count0][count1] = Final[count1]
            else :
                BAMDS[count0][count1] = np.nan
            count1 = count1 + 1
        count0 = count0 + 1

    #Gets and Interpolates the Error for each band
    u_error = np.loadtxt( 'u_err.csv' , skiprows = 1 , delimiter = ',' , unpack = 1 , dtype = 'float' )
    u_x = u_error[0]
    u_y = u_error[1]
    u_error_interpol = interpolate.interp1d( u_x , u_y )( a )


    g_error = np.loadtxt( 'g_err.csv' , skiprows = 1 , delimiter = ',' , unpack = 1 , dtype = 'float' )
    g_x = g_error[0]
    g_y = g_error[1]
    g_error_interpol = interpolate.interp1d( g_x , g_y )( b )

    i_error = np.loadtxt( 'i_err.csv' , skiprows = 1 , delimiter = ',' , unpack = 1 , dtype = 'float' )
    i_x = i_error[0]
    i_y = i_error[1]
    i_error_interpol = interpolate.interp1d( i_x , i_y )( d )

    z_error = np.loadtxt( 'z_err.csv' , skiprows = 1 , delimiter = ',' , unpack = 1 , dtype = 'float' )
    z_x = z_error[0]
    z_y = z_error[1]
    z_error_interpol = interpolate.interp1d( z_x , z_y )( e )

    r_error_interpol = np.ones_like(z_error_interpol)

    error_interpol = [u_error_interpol, g_error_interpol, r_error_interpol, i_error_interpol, z_error_interpol]

    #Adds the Error to the data
    BAMDS_noise = [ [] , [] , [], [] , [] ]

    count = 0
    while count < len(BAMDS_noise):
        count1 = 0
        while count1 < BAMDS[count].size:
            BAMDS_noise[count] = np.append(BAMDS_noise[count], np.random.normal(BAMDS[count][count1], (error_interpol[count])[count1]))
            count1 += 1
        count += 1


    #Creates and Returns the Figure and CSV File
    a_noise = BAMDS_noise[0]
    b_noise = BAMDS_noise[1]
    c_noise = BAMDS_noise[2]
    d_noise = BAMDS_noise[3]
    e_noise = BAMDS_noise[4]

    fig = plt.figure()

    plt.figure( figsize = ( 3.2 , 3.2 ) , dpi = 200 )

    plt.suptitle( DATA[3:30] + " , " + str(select) + " , " + str(YearShift) , x = 0.5 , y = 0.94 , fontsize = 10 )

    ax1 = plt.subplot2grid( ( 2 , 1 ) , ( 0 , 0 ) )
    ax2 = plt.subplot2grid( ( 2 , 1 ) , ( 1 , 0 ) )

    ax1.scatter( mjd , u , marker = 's' , s = 0.2 , color = '#0000FF' , linewidths = 0 )
    ax1.scatter( mjd , g , marker = 's' , s = 0.2 , color = '#00CF00' , linewidths = 0 )
    ax1.scatter( mjd , r , marker = 's' , s = 0.2 , color = '#FF0000' , linewidths = 0 )
    ax1.scatter( mjd , i , marker = 's' , s = 0.2 , color = '#FFFF00' , linewidths = 0 )
    ax1.scatter( mjd , z , marker = 's' , s = 0.2 , color = '#000000' , linewidths = 0 )

    ax1.scatter( Target_mjd , Target_u , marker = '.' , s = 10 , color = '#0000FF' , linewidths = 0 , )
    ax1.scatter( Target_mjd , Target_g , marker = '.' , s = 10 , color = '#00CF00' , linewidths = 0 , )
    ax1.scatter( Target_mjd , Target_i , marker = '.' , s = 10 , color = '#FFFF00' , linewidths = 0 , )
    ax1.scatter( Target_mjd , Target_z , marker = '.' , s = 10 , color = '#000000' , linewidths = 0 , )

    ax1.invert_yaxis()

    ax1.set_xlim( -365.25/2 , 5 * 365.25 + 365.25/2 )
    ax1.set_ylim( np.mean(MAG) + 15 * np.std(MAG) , np.mean(MAG) - 15 * np.std(MAG) )
    ax1.set_xticks( [ 0*365.25 , 1*365.25 , 2*365.25 , 3*365.25 , 4*365.25 , 5*365.25 , 6*365.25, 7*365.25] )

    ax2.scatter( Final_Dates , a_noise , marker = 'o' , s = 10 , color = '#0000FF' , linewidths = 0 )
    ax2.scatter( Final_Dates , b_noise , marker = 'o' , s = 10 , color = '#00CF00' , linewidths = 0 )
    ax2.scatter( Final_Dates , d_noise , marker = 'o' , s = 10 , color = '#FFFF00' , linewidths = 0 )
    ax2.scatter( Final_Dates , e_noise , marker = 'o' , s = 10 , color = '#000000' , linewidths = 0 )

    ax2.invert_yaxis()

    ax2.set_xlim( -365.25/2 , 5 * 365.25 + 365.25/2 )
    ax2.set_ylim( np.mean(Final) + 5 * np.std(MAG) , np.mean(Final) - 5 * np.std(MAG) )
    ax2.set_xticks( [ 0*365.25 , 1*365.25 , 2*365.25 , 3*365.25 , 4*365.25 , 5*365.25 , 6*365.25, 7*365.25 ] )

    BAMD = ['u', 'g', 'r', 'i', 'z']

    Final1 = []
    for i in range(len(Final_Dates)):
        Final1.append([])

    for i in range(len(BAMDS_noise)):
        for j in range(len(BAMDS_noise[i])):
            if ~np.isnan(BAMDS_noise[i][j]):
                Final1[j] = [Final_Dates[j], BAMDS_noise[i][j], BAMD[i]]

    if (Target_dates.size != len(Final1)):
        return ""

    Final1.insert(0, ["time(MJD)", "mag", "band"])


    return plt.savefig("dee" + str(D_Num) + "/" + "'" + DATA[3:30] + " , " + str(select) + " , " + str(YearShift) + "'.jpg"), np.savetxt("de" + str(D_Num) + "/" + "'" + DATA[3:30] + " , " + str(select) + " , " + str(YearShift) + "'.csv", Final1, fmt='%s', delimiter=', ')

In [ ]:
#Code that runs FUNC, inputting many different values
for D_Num in range(4):
    output = np.loadtxt( 'TargetData.csv' , skiprows = 1 , delimiter = ',' , unpack = 1 , dtype = 'str' )
    Target_ID = output[0].astype('int')

    DATALOOP = np.loadtxt( 'Quasars.csv' , skiprows = 1 , delimiter = ',' , unpack = 1 , dtype = 'str' )[D_Num]

    IDLOOP = np.unique(Target_ID)

    [ "D1" , "D2" , "D3" , "D4" ]
    OppDates = [ OppDateFind(2,25,59) , OppDateFind(10,00,28) , OppDateFind(14,19,27) , OppDateFind(22,15,31) ]
    OppDate = OppDates[D_Num]

    D_Num = D_Num + 1

    for A in DATALOOP :
        if A != "":
            DATA = "D" + str(D_Num) + "/" + A + ".csv"
            INTERPOL, mjd3, YearMedian, MonthMedian, mjd, Bands, MAG = interpQuasar(DATA, OppDate)
            [ u , g , r , i , z ] = Bands
            [ u_med , g_med , r_med , i_med , z_med ] = MonthMedian
            for select in np.unique(Target_ID) :
                for YearShift in np.linspace( -3, 7, num = 11):
                    try:
                        FUNC( DATA , select , YearShift , OppDate, D_Num, i)
                    except ValueError :

                        1 == 1